# Preprocessing: step 1
data0_raw->data1_filtered_imputed
- rename columns
- imputation
- remove patients who are ventilated within 6 hours of ICU admission

In [2]:
#python mimic_direct_extract.py  --min_age 18 --min_duration 6 --max_duration 99999 --psql_host "/var/run/postgresql/"  --out_path $PATH/data/mimic_raw/mimic_raw.h5  --extract_notes 0 --plot_hist 0
import numpy as np
import pandas as pd
import sys
import os
#DATAFILE_raw = '../../MIMIC_Extract/data/220804/all_hourly_data.h5'
DATAFILE_raw = '../data/mimic_raw/mimic_raw.h5'
X = pd.read_hdf(DATAFILE_raw,'vitals_labs')
Y = pd.read_hdf(DATAFILE_raw,'interventions')
static = pd.read_hdf(DATAFILE_raw,'patients')
Y = Y[['vent']]

FILLNA_CODE = 4
AVERAGE_IMPUTE=False

print(len(X.reset_index()['subject_id'].unique()))
print(Y.sum())
print(len(Y))
print(Y.sum()/len(Y))
Y.head()

38217
vent    1336104
dtype: int64
3766419
vent    0.354741
dtype: float64


vent
subject_id hadm_id icustay_id hours_in      
3          145834  211552     0            1
                              1            1
                              2            1
                              3            1
                              4            1

In [3]:
X.columns

MultiIndex([(             'bicarbonate', 'count'),
            (             'bicarbonate', 'first'),
            (             'bicarbonate',   'max'),
            (             'bicarbonate',  'mean'),
            (             'bicarbonate',   'min'),
            (             'bicarbonate',   'std'),
            ('diastolic blood pressure', 'count'),
            ('diastolic blood pressure', 'first'),
            ('diastolic blood pressure',   'max'),
            ('diastolic blood pressure',  'mean'),
            ...
            (  'white blood cell count',   'max'),
            (  'white blood cell count',  'mean'),
            (  'white blood cell count',   'min'),
            (  'white blood cell count',   'std'),
            (                      'ph', 'count'),
            (                      'ph', 'first'),
            (                      'ph',   'max'),
            (                      'ph',  'mean'),
            (                      'ph',   'min'),
            (  

In [4]:
#extract & rename columns
_vital = ['respiratory rate','heart rate','spo2','systolic blood pressure', 'diastolic blood pressure', 'temperature']
_CBC = ['hematocrit','hemoglobin','platelets','white blood cell count','red blood cell count', 'mean corpuscular hemoglobin', \
'mean corpuscular hemoglobin concentration', 'mean corpuscular volume']
_ABGA = ['ph', 'bicarbonate', 'partial pressure of oxygen','partial pressure of carbon dioxide','oxygen saturation', 'fraction inspired oxygen']
_cols = _vital+_CBC+_ABGA

vital = ['rr','hr','spo2','sbp','dbp','temp']
CBC = ['hct','hb','platelets','wbc','rbc','mch','mchc','mcv']
ABGA = ['ph','hco3', 'po2','pco2','sao2','fio2']
cols = vital+CBC+ABGA

X_=X.loc[:,(_cols, ['mean','first','max','min'])]
for i in range(len(cols)):
    X_.rename(columns={_cols[i]:cols[i]}, inplace = True)
X_.head()

<ipython-input-4-0fffc7448100>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_.rename(columns={_cols[i]:cols[i]}, inplace = True)
<ipython-input-4-0fffc7448100>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_.rename(columns={_cols[i]:cols[i]}, inplace = True)
<ipython-input-4-0fffc7448100>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_.rename(columns={_cols[i]:cols[i]}, inplace = True)
<ipython-input-4-0fffc7448100>:15:

LEVEL2                                         rr                       hr  \
Aggregation Function                         mean first   max  min    mean   
subject_id hadm_id icustay_id hours_in                                       
3          145834  211552     0         16.000000   8.0  24.0  8.0  143.00   
                              1         15.500000  21.0  21.0  5.0  153.75   
                              2          7.000000   0.0  21.0  0.0  137.75   
                              3          5.250000   0.0  16.0  0.0  129.25   
                              4         13.666667   3.0  19.0  3.0  146.00   

LEVEL2                                                        spo2         \
Aggregation Function                    first    max    min   mean  first   
subject_id hadm_id icustay_id hours_in                                      
3          145834  211552     0         143.0  151.0  135.0  74.00   74.0   
                              1         168.0  168.0  135.0    NaN    NaN   
                              2         144.0  145.0  128.0  98.25   99.0   
                              3         129.0  143.0  121.0  99.75  100.0   
                              4         146.0  146.0  146.0  98.00   96.0   

LEVEL2                                  ...  pco2        sao2              \
Aggregation Function                    ...   max   min  mean first   max   
subject_id hadm_id icustay_id hours_in  ...                                 
3          145834  211552     0         ...  40.0  28.0   NaN   NaN   NaN   
                              1         ...   NaN   NaN   NaN   NaN   NaN   
                              2         ...  33.0  33.0  99.0  99.0  99.0   
                              3         ...   NaN   NaN   NaN   NaN   NaN   
                              4         ...   NaN   NaN   NaN   NaN   NaN   

LEVEL2                                       fio2                  
Aggregation Function                     min mean first  max  min  
subject_id hadm_id icustay_id hours_in                             
3          145834  211552     0          NaN  1.0   1.0  1.0  1.0  
                              1          NaN  NaN   NaN  NaN  NaN  
                              2         99.0  NaN   NaN  NaN  NaN  
                              3          NaN  NaN   NaN  NaN  NaN  
                              4          NaN  0.8   1.0  1.0  0.6  

[5 rows x 80 columns]

In [5]:
#impute with ffill & bfill
ID_COLS = ['subject_id', 'hadm_id', 'icustay_id']

print(X_.isna().sum().sum())
if FILLNA_CODE == 0:
    X_=X_.groupby(ID_COLS).fillna(method='ffill').groupby(ID_COLS).fillna(method='bfill')
if FILLNA_CODE == 1:
    X_=X_.groupby(ID_COLS).fillna(method='ffill').groupby(ID_COLS).fillna(method='bfill',limit=6)
if FILLNA_CODE == 2:
    X_=X_.groupby(ID_COLS).fillna(method='ffill')
if FILLNA_CODE == 3:
    print("bfill6h and drop")
    print(X_.shape)
    print(X_.isna().sum().sum())
    X_=X_.groupby(ID_COLS).fillna(method='ffill').groupby(ID_COLS).fillna(method='bfill',limit=6)
    temp = X_.copy()
    print(X_.isna().sum().sum())
    X_=X_.dropna(axis=0,thresh=80-4*5)
    print(X_.isna().sum().sum())
    print(X_.shape)
if FILLNA_CODE == 4:
    print("Drop and reffill")
    print(X_.shape)
    print(X_.isna().sum().sum())
    X_=X_.groupby(ID_COLS).fillna(method='ffill')
    temp = X_.copy()
    print(X_.isna().sum().sum())
    X_=X_.dropna(axis=0,thresh=80-4*5)
    print(X_.isna().sum().sum())
    print(X_.shape)
X_.head()

212529400
Drop and reffill
(3766419, 80)
212529400
21939392
15921040
(3620050, 80)


LEVEL2                                         rr                       hr  \
Aggregation Function                         mean first   max  min    mean   
subject_id hadm_id icustay_id hours_in                                       
3          145834  211552     0         16.000000   8.0  24.0  8.0  143.00   
                              1         15.500000  21.0  21.0  5.0  153.75   
                              2          7.000000   0.0  21.0  0.0  137.75   
                              3          5.250000   0.0  16.0  0.0  129.25   
                              4         13.666667   3.0  19.0  3.0  146.00   

LEVEL2                                                        spo2         \
Aggregation Function                    first    max    min   mean  first   
subject_id hadm_id icustay_id hours_in                                      
3          145834  211552     0         143.0  151.0  135.0  74.00   74.0   
                              1         168.0  168.0  135.0  74.00   74.0   
                              2         144.0  145.0  128.0  98.25   99.0   
                              3         129.0  143.0  121.0  99.75  100.0   
                              4         146.0  146.0  146.0  98.00   96.0   

LEVEL2                                  ...  pco2        sao2              \
Aggregation Function                    ...   max   min  mean first   max   
subject_id hadm_id icustay_id hours_in  ...                                 
3          145834  211552     0         ...  40.0  28.0   NaN   NaN   NaN   
                              1         ...  40.0  28.0   NaN   NaN   NaN   
                              2         ...  33.0  33.0  99.0  99.0  99.0   
                              3         ...  33.0  33.0  99.0  99.0  99.0   
                              4         ...  33.0  33.0  99.0  99.0  99.0   

LEVEL2                                       fio2                  
Aggregation Function                     min mean first  max  min  
subject_id hadm_id icustay_id hours_in                             
3          145834  211552     0          NaN  1.0   1.0  1.0  1.0  
                              1          NaN  1.0   1.0  1.0  1.0  
                              2         99.0  1.0   1.0  1.0  1.0  
                              3         99.0  1.0   1.0  1.0  1.0  
                              4         99.0  0.8   1.0  1.0  0.6  

[5 rows x 80 columns]

In [6]:
temp
#print(f"Original nan count:", temp.isna().sum().sum())
print(f"Original #patients: ", temp.shape)

for i in range(15):
    temp_=temp.dropna(axis=0,thresh=80-4*i,inplace=False)
    print()
    print(f"drop row with more than {i} NaNs")
#    print(f"remaining nan", temp_.isna().sum().sum())
    print(f"remaining #observation: ",temp_.shape[0])


Original #patients:  (3766419, 80)

drop row with more than 0 NaNs
remaining #observation:  2017091

drop row with more than 1 NaNs
remaining #observation:  2847033

drop row with more than 2 NaNs
remaining #observation:  3026943

drop row with more than 3 NaNs
remaining #observation:  3050189

drop row with more than 4 NaNs
remaining #observation:  3178734

drop row with more than 5 NaNs
remaining #observation:  3620050

drop row with more than 6 NaNs
remaining #observation:  3644087

drop row with more than 7 NaNs
remaining #observation:  3670576

drop row with more than 8 NaNs
remaining #observation:  3685626

drop row with more than 9 NaNs
remaining #observation:  3693795

drop row with more than 10 NaNs
remaining #observation:  3701037

drop row with more than 11 NaNs
remaining #observation:  3714901

drop row with more than 12 NaNs
remaining #observation:  3719605

drop row with more than 13 NaNs
remaining #observation:  3729731

drop row with more than 14 NaNs
remaining #observa

In [7]:
len(X_.reset_index()['subject_id'].unique())

37569

In [8]:
if AVERAGE_IMPUTE:
    #impute with ffill & bfill
    ID_COLS = ['subject_id', 'hadm_id', 'icustay_id']

    X_aveimpute = X_.copy()
    idx = pd.IndexSlice

    print(X_aveimpute.isna().sum().sum())
    icustay_means = X_aveimpute.loc[:, idx[:, ['first', 'max', 'mean', 'min']]].groupby(ID_COLS).mean()
    global_means = X_aveimpute.loc[:, idx[:, ['first', 'max', 'mean', 'min']]].mean(axis=0)
    X_aveimpute=X_aveimpute.groupby(ID_COLS).fillna(icustay_means).fillna(global_means)
    #X_aveimpute=X_.fillna(global_means)
    print(X_aveimpute.isna().sum().sum())
    X_aveimpute.head()

In [9]:
#for patient with no lab test
#imputation with predefined normal value for each variable

reference_value = {      # average
#vital
    'rr': 16,           # 19.75339
    'hr':80,            # 86.15997
    'spo2': 97,      # 90.743683
    'sbp':110,          # 122.7326
    'dbp':70,           # 60.76322
    'temp':36.5,        # 37.0867
#CBC
    'hct': 41.5,        # 30.491288
    'hb': 13.5,         # 10.396213
    'platelets':275,    # 219.158934
    'wbc': 7,           # 12.412393
    'rbc': 4.45,        # 3.454153
    'mch': 29.5,        # 30.333704
    'mchc': 33.25,      # 33.850937
    'mcv': 90,          # 89.702025
#ABGA
    'ph': 7.4,          # 7.388153
    'hco3': 24,         # 24.595758
    'po2': 87.5,        # 130.492106
    'pco2': 40,         # 41.616208
    'sao2': 97,     # 97.09498
    'fio2' : 0.21       # 0.512136  
}
print(X_.isna().sum().sum())
X_.fillna(
    pd.Series(X_.columns.get_level_values(0).map(reference_value.get), X_.columns), inplace=True
)
print(X_.isna().sum().sum())
X_.head()

15921040
0


LEVEL2                                         rr                       hr  \
Aggregation Function                         mean first   max  min    mean   
subject_id hadm_id icustay_id hours_in                                       
3          145834  211552     0         16.000000   8.0  24.0  8.0  143.00   
                              1         15.500000  21.0  21.0  5.0  153.75   
                              2          7.000000   0.0  21.0  0.0  137.75   
                              3          5.250000   0.0  16.0  0.0  129.25   
                              4         13.666667   3.0  19.0  3.0  146.00   

LEVEL2                                                        spo2         \
Aggregation Function                    first    max    min   mean  first   
subject_id hadm_id icustay_id hours_in                                      
3          145834  211552     0         143.0  151.0  135.0  74.00   74.0   
                              1         168.0  168.0  135.0  74.00   74.0   
                              2         144.0  145.0  128.0  98.25   99.0   
                              3         129.0  143.0  121.0  99.75  100.0   
                              4         146.0  146.0  146.0  98.00   96.0   

LEVEL2                                  ...  pco2        sao2              \
Aggregation Function                    ...   max   min  mean first   max   
subject_id hadm_id icustay_id hours_in  ...                                 
3          145834  211552     0         ...  40.0  28.0  97.0  97.0  97.0   
                              1         ...  40.0  28.0  97.0  97.0  97.0   
                              2         ...  33.0  33.0  99.0  99.0  99.0   
                              3         ...  33.0  33.0  99.0  99.0  99.0   
                              4         ...  33.0  33.0  99.0  99.0  99.0   

LEVEL2                                       fio2                  
Aggregation Function                     min mean first  max  min  
subject_id hadm_id icustay_id hours_in                             
3          145834  211552     0         97.0  1.0   1.0  1.0  1.0  
                              1         97.0  1.0   1.0  1.0  1.0  
                              2         99.0  1.0   1.0  1.0  1.0  
                              3         99.0  1.0   1.0  1.0  1.0  
                              4         99.0  0.8   1.0  1.0  0.6  

[5 rows x 80 columns]

In [10]:
#no observation after 10 days
def filter_by_hour(df,max_hours):
    hours_in = df.index.get_level_values(3)
    condition = hours_in <= max_hours # 10days
    df=df[condition]
    return df

if AVERAGE_IMPUTE:
    X_aveimpute=filter_by_hour(X_aveimpute,240)
X_=filter_by_hour(X_,240)
X_.head()

LEVEL2                                         rr                       hr  \
Aggregation Function                         mean first   max  min    mean   
subject_id hadm_id icustay_id hours_in                                       
3          145834  211552     0         16.000000   8.0  24.0  8.0  143.00   
                              1         15.500000  21.0  21.0  5.0  153.75   
                              2          7.000000   0.0  21.0  0.0  137.75   
                              3          5.250000   0.0  16.0  0.0  129.25   
                              4         13.666667   3.0  19.0  3.0  146.00   

LEVEL2                                                        spo2         \
Aggregation Function                    first    max    min   mean  first   
subject_id hadm_id icustay_id hours_in                                      
3          145834  211552     0         143.0  151.0  135.0  74.00   74.0   
                              1         168.0  168.0  135.0  74.00   74.0   
                              2         144.0  145.0  128.0  98.25   99.0   
                              3         129.0  143.0  121.0  99.75  100.0   
                              4         146.0  146.0  146.0  98.00   96.0   

LEVEL2                                  ...  pco2        sao2              \
Aggregation Function                    ...   max   min  mean first   max   
subject_id hadm_id icustay_id hours_in  ...                                 
3          145834  211552     0         ...  40.0  28.0  97.0  97.0  97.0   
                              1         ...  40.0  28.0  97.0  97.0  97.0   
                              2         ...  33.0  33.0  99.0  99.0  99.0   
                              3         ...  33.0  33.0  99.0  99.0  99.0   
                              4         ...  33.0  33.0  99.0  99.0  99.0   

LEVEL2                                       fio2                  
Aggregation Function                     min mean first  max  min  
subject_id hadm_id icustay_id hours_in                             
3          145834  211552     0         97.0  1.0   1.0  1.0  1.0  
                              1         97.0  1.0   1.0  1.0  1.0  
                              2         99.0  1.0   1.0  1.0  1.0  
                              3         99.0  1.0   1.0  1.0  1.0  
                              4         99.0  0.8   1.0  1.0  0.6  

[5 rows x 80 columns]

In [11]:
# #remove data observed after MV
# first_mv_idx_MV = extract_first_mv_idx(Y)
# X_filter_=filter_after_MV(X_filter.reset_index(),first_mv_idx_MV)

# #filter patients with less than 6 observation
# X_filter_6h, Y_6h, static_6h = filter_earlyexit(X_filter_,Y,static,6)
# X_filter_6h.head()

# #remove data observed after MV
# X_aveimpute=filter_after_MV(X_aveimpute.reset_index(),first_mv_idx_MV)

# #filter patients with less than 6 observation
# X_aveimpute, Y_6h, static_6h = filter_earlyexit(X_aveimpute,Y,static,6)
# X_aveimpute.head()

# #list of subject ID remaining
# SID_list = X_filter_6h['subject_id'].reset_index(drop=True).unique()
# print(SID_list.shape)

# #remove patients without data from Y, statistics
# Y_6h=Y_6h.reset_index(drop=True)
# Y_6h=Y_6h[Y_6h['subject_id'].isin(SID_list)]

# static_6h=static_6h.reset_index(drop=True)
# static_6h=static_6h[static_6h['subject_id'].isin(SID_list)]

# static_6h['MV_ICU'] =0
# static_6h.loc[static_6h['subject_id'].isin(first_mv_idx_MV.index),'MV_ICU']=1

In [12]:
#remove data observed after MV
#filter patients with less than 6 observation
vent_time = Y[Y['vent']==1].reset_index()[['subject_id','hours_in']].groupby('subject_id').min().reset_index()
early_exit_id = vent_time[vent_time['hours_in']<6]['subject_id'].unique()

X_=X_.reset_index()
Y=Y.reset_index()
static=static.reset_index()

X_ = X_[~X_['subject_id'].isin(early_exit_id)]

if AVERAGE_IMPUTE:
    X_aveimpute_=X_aveimpute.reset_index()
    X_aveimpute= X_aveimpute_[~X_aveimpute_['subject_id'].isin(early_exit_id)]


#remove early exit_id
vent_time = vent_time[vent_time['hours_in']>6].reset_index(drop=True)
vent_time.columns = ['subject_id','mvtime']

In [13]:
mvid = vent_time['subject_id'].unique()
idx_to_remove = pd.Int64Index([])

# MV 이후 observation들의 index 추출
for i in mvid:
    mvtime = vent_time[vent_time['subject_id']==i]['mvtime'].item()
    idx  = X_[(X_['subject_id']==i)&(X_['hours_in']>=mvtime)].index
    idx_to_remove = idx_to_remove.append(idx)

# MV 이후 observation 삭제하기
X_ = X_[~X_.index.isin(idx_to_remove)]
if AVERAGE_IMPUTE:
    X_aveimpute = X_aveimpute[~X_aveimpute.index.isin(idx_to_remove)]


# 제거된 환자 y, static에 반영하기
ids = X_['subject_id'].unique()
Y_6h = Y[Y['subject_id'].isin(ids)]
static_6h = static[static['subject_id'].isin(ids)]

static_6h['MV_ICU'] =0
static_6h.loc[static_6h['subject_id'].isin(mvid),'MV_ICU']=1
static_6h['MV_ICU'].describe()



<ipython-input-13-c3b0a4944d0a>:2: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  idx_to_remove = pd.Int64Index([])
<ipython-input-13-c3b0a4944d0a>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  static_6h['MV_ICU'] =0


count    24940.000000
mean         0.131756
std          0.338232
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: MV_ICU, dtype: float64

In [14]:
#MIMIC은 89세 이상의 경우 300으로 처리, 보정을 위해서 210 빼줘야함
static_6h.loc[static_6h['age']>100,'age']-=210
static_6h[static_6h['age']>100]

static_6h['MV_ICU'].value_counts()

# input으로 같이 넣어줄 static 데이터
static_to_keep_6h = static_6h[['subject_id','icustay_id','hadm_id','gender', 'age', 'intime']]
static_to_keep_6h.loc[:, 'intime'] = static_to_keep_6h['intime'].astype('datetime64').apply(lambda x : x.hour)
static_to_keep_6h = pd.get_dummies(static_to_keep_6h, columns = ['gender'])
static_to_keep_6h = static_to_keep_6h
static_to_keep_6h.head()

<ipython-input-14-7793aaee40f8>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  static_to_keep_6h.loc[:, 'intime'] = static_to_keep_6h['intime'].astype('datetime64').apply(lambda x : x.hour)


,subject_id,icustay_id,hadm_id,age,intime,gender_F,gender_M
1,4,294638,185777,47.845047,0,1,0
2,6,228232,107064,65.942297,21,1,0
4,11,229441,194540,50.148295,6,1,0
6,13,263738,143045,39.866118,18,1,0
8,18,298129,188822,50.841559,11,0,1


In [15]:
if FILLNA_CODE == 0:
    DATAFILE_filtered = '../data/mimic_raw/mimic_worstimputed.h5'
if FILLNA_CODE == 1:
    DATAFILE_filtered = '../data/mimic_raw/mimic_bfill6himputed.h5'
if FILLNA_CODE == 2:
    DATAFILE_filtered = '../data/mimic_raw/mimic_nobfillimputed.h5'    
if FILLNA_CODE == 3:
    DATAFILE_filtered = '../data/mimic_raw/mimic_bfill6h_drop_imputed.h5'    
if FILLNA_CODE == 4:
    DATAFILE_filtered = '../data/mimic_raw/mimic_dropref_imputed.h5'



X_.to_hdf(DATAFILE_filtered,'vitals_labs')
Y_6h.to_hdf(DATAFILE_filtered,'interventions')
static_6h.to_hdf(DATAFILE_filtered,'patients', format='table')
static_to_keep_6h.to_hdf(DATAFILE_filtered,'patients_to_keep', format='table')

/usr/local/lib/python3.8/dist-packages/tables/attributeset.py:459: NaturalNameWarning: object name is not a valid Python identifier: 'axis0_nameAggregation Function'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/usr/local/lib/python3.8/dist-packages/tables/attributeset.py:459: NaturalNameWarning: object name is not a valid Python identifier: 'block0_items_nameAggregation Function'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/usr/local/lib/python3.8/dist-packages/tables/attributeset.py:459: NaturalNameWarning: object name is not a valid Python identifier: 'block1_items_nameAggregation Function'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use

In [16]:
if AVERAGE_IMPUTE:
    DATAFILE_mean_impute = '../data/mimic_raw/mimic_meanimputed.h5'

    X_aveimpute.to_hdf(DATAFILE_mean_impute,'vitals_labs')
    Y_6h.to_hdf(DATAFILE_mean_impute,'interventions')
    static_6h.to_hdf(DATAFILE_mean_impute,'patients', format='table')
    static_to_keep_6h.to_hdf(DATAFILE_mean_impute,'patients_to_keep', format='table')

# Preprocessing: step 2
data1_filtered_imputed ==> data2_(all,micu)_(worst,mean,first)
- split dataset -> hourly mean, first, worst
- apply worst value algorithm for worst dataset


In [17]:
import numpy as np
import pandas as pd
import sys
import os

use_mean_impute = 0

if use_mean_impute == 0:
    if FILLNA_CODE == 0:
        DATAFILE_filtered = '../data/mimic_raw/mimic_worstimputed.h5'
    if FILLNA_CODE == 1:
        DATAFILE_filtered = '../data/mimic_raw/mimic_bfill6himputed.h5'
    if FILLNA_CODE == 2:
        DATAFILE_filtered = '../data/mimic_raw/mimic_nobfillimputed.h5'   
    if FILLNA_CODE == 3:
        DATAFILE_filtered = '../data/mimic_raw/mimic_bfill6h_drop_imputed.h5'    
    if FILLNA_CODE == 4:
        DATAFILE_filtered = '../data/mimic_raw/mimic_dropref_imputed.h5'
else:
    DATAFILE_filtered = '../data/mimic_raw/mimic_meanimputed.h5'

 



X = pd.read_hdf(DATAFILE_filtered,'vitals_labs')
Y = pd.read_hdf(DATAFILE_filtered,'interventions')
static = pd.read_hdf(DATAFILE_filtered,'patients')
static_to_keep = pd.read_hdf(DATAFILE_filtered,'patients_to_keep')

vital = ['rr','hr','spo2','sbp','dbp','temp']
CBC = ['hct','hb','platelets','wbc','rbc','mch','mchc','mcv']
ABGA = ['ph','hco3', 'po2','pco2','sao2','fio2']
cols = vital+CBC+ABGA
ID_COLS = ['subject_id', 'hadm_id', 'icustay_id', 'hours_in']
X=X.set_index(ID_COLS)
X.head()

LEVEL2                                    rr                      hr        \
Aggregation Function                    mean first   max   min  mean first   
subject_id hadm_id icustay_id hours_in                                       
4          185777  294638     5         16.0  16.0  16.0  16.0  82.0  82.0   
                              6         18.0  18.0  18.0  18.0  81.0  81.0   
                              7         24.0  24.0  24.0  24.0  80.0  80.0   
                              8         24.0  24.0  24.0  24.0  85.0  85.0   
                              9         24.0  24.0  24.0  24.0  92.0  92.0   

LEVEL2                                               spo2         ...  pco2  \
Aggregation Function                     max   min   mean  first  ...   max   
subject_id hadm_id icustay_id hours_in                            ...         
4          185777  294638     5         82.0  82.0   95.0   95.0  ...  28.0   
                              6         81.0  81.0  100.0  100.0  ...  28.0   
                              7         80.0  80.0  100.0  100.0  ...  28.0   
                              8         85.0  85.0   91.0   91.0  ...  28.0   
                              9         92.0  92.0   98.0   98.0  ...  28.0   

LEVEL2                                        sao2                    fio2  \
Aggregation Function                     min  mean first   max   min  mean   
subject_id hadm_id icustay_id hours_in                                       
4          185777  294638     5         28.0  94.0  94.0  94.0  94.0  0.95   
                              6         28.0  94.0  94.0  94.0  94.0  0.95   
                              7         28.0  94.0  94.0  94.0  94.0  0.50   
                              8         28.0  94.0  94.0  94.0  94.0  0.50   
                              9         28.0  94.0  94.0  94.0  94.0  0.50   

LEVEL2                                                    
Aggregation Function                   first   max   min  
subject_id hadm_id icustay_id hours_in                    
4          185777  294638     5         0.95  0.95  0.95  
                              6         0.95  0.95  0.95  
                              7         0.50  0.50  0.50  
                              8         0.50  0.50  0.50  
                              9         0.50  0.50  0.50  

[5 rows x 80 columns]

In [18]:
print(len(X.reset_index()['subject_id'].unique()))
print(Y['vent'].sum())
print(len(Y))
print(len(X))
print(Y['vent'].sum()/len(Y['vent']))
Y.head()

24940
442314
2163240
1347625
0.2044682975536695


,subject_id,hadm_id,icustay_id,hours_in,vent
146,4,185777,294638,0,0
147,4,185777,294638,1,0
148,4,185777,294638,2,0
149,4,185777,294638,3,0
150,4,185777,294638,4,0


In [19]:
reference_value = {      # average
#vital
    'rr': 16,           # 19.75339
    'hr':80,            # 86.15997
    'spo2': 97,      # 90.743683
    'sbp':110,          # 122.7326
    'dbp':70,           # 60.76322
    'temp':36.5,        # 37.0867
#CBC
    'hct': 41.5,        # 30.491288
    'hb': 13.5,         # 10.396213
    'platelets':275,    # 219.158934
    'wbc': 7,           # 12.412393
    'rbc': 4.45,        # 3.454153
    'mch': 29.5,        # 30.333704
    'mchc': 33.25,      # 33.850937
    'mcv': 90,          # 89.702025
#ABGA
    'ph': 7.4,          # 7.388153
    'hco3': 24,         # 24.595758
    'po2': 87.5,        # 130.492106
    'pco2': 40,         # 41.616208
    'sao2': 97,     # 97.09498
    'fio2' : 0.21       # 0.512136  
}

idx = pd.IndexSlice
X_worst = X.loc[:,idx[cols, ['min','max']]].copy()
cnt=0
for k in X_worst.columns:
    #for min, max calc distance from reference value
    cnt=cnt+1
    if cnt%2==0:
        continue
    
    ref = reference_value[k[0]]
    print(k,cnt,ref)
    min_diff = abs(X_worst[k[0],'min'] - reference_value[k[0]] )
    max_diff = abs(X_worst[k[0],'max'] - reference_value[k[0]] )
    
    X_worst[k[0],'worst'] = np.where(max_diff>min_diff,
                    X_worst[k[0],'max'], X_worst[k[0],'min'])


X_worst = X_worst.loc[:,idx[:, ['worst']]]
X_worst.columns = X_worst.columns.droplevel(-1)
X_worst.columns.name = None
X_worst.head()

('rr', 'min') 1 16
('hr', 'min') 3 80
('spo2', 'min') 5 97
('sbp', 'min') 7 110
('dbp', 'min') 9 70
('temp', 'min') 11 36.5
('hct', 'min') 13 41.5
('hb', 'min') 15 13.5
('platelets', 'min') 17 275
('wbc', 'min') 19 7
('rbc', 'min') 21 4.45
('mch', 'min') 23 29.5
('mchc', 'min') 25 33.25
('mcv', 'min') 27 90
('ph', 'min') 29 7.4
('hco3', 'min') 31 24
('po2', 'min') 33 87.5
('pco2', 'min') 35 40
('sao2', 'min') 37 97
('fio2', 'min') 39 0.21


rr    hr   spo2    sbp   dbp  \
subject_id hadm_id icustay_id hours_in                                   
4          185777  294638     5         16.0  82.0   95.0  101.0  61.0   
                              6         18.0  81.0  100.0   97.0  55.0   
                              7         24.0  80.0  100.0  109.0  58.0   
                              8         24.0  85.0   91.0  110.0  56.0   
                              9         24.0  92.0   98.0  110.0  60.0   

                                             temp        hct    hb  platelets  \
subject_id hadm_id icustay_id hours_in                                          
4          185777  294638     5         37.444445  31.799999  10.6      201.0   
                              6         37.444445  31.799999  10.6      201.0   
                              7         36.666698  31.799999  10.6      201.0   
                              8         36.666698  31.799999  10.6      201.0   
                              9         36.666698  31.799999  10.6      201.0   

                                        wbc   rbc   mch  mchc   mcv    ph  \
subject_id hadm_id icustay_id hours_in                                      
4          185777  294638     5         9.7  3.42  30.8  33.2  93.0  7.47   
                              6         9.7  3.42  30.8  33.2  93.0  7.47   
                              7         9.7  3.42  30.8  33.2  93.0  7.47   
                              8         9.7  3.42  30.8  33.2  93.0  7.47   
                              9         9.7  3.42  30.8  33.2  93.0  7.47   

                                        hco3   po2  pco2  sao2  fio2  
subject_id hadm_id icustay_id hours_in                                
4          185777  294638     5         21.0  74.0  28.0  94.0  0.95  
                              6         21.0  74.0  28.0  94.0  0.95  
                              7         21.0  74.0  28.0  94.0  0.50  
                              8         21.0  74.0  28.0  94.0  0.50  
                              9         21.0  74.0  28.0  94.0  0.50

In [20]:
#split dataset --> mean / first / worst
def split_data(X,target):
    idx = pd.IndexSlice
    X_=X.loc[:,idx[cols, target]]
    X_.columns = X_.columns.droplevel(-1)
    X_.columns.name = None
    return X_
X_mean = split_data(X,'mean')
X_first = split_data(X,'first')

del X

In [21]:
dataset = [X_worst,X_mean,X_first]

In [22]:
# pfratio
for idx, df in enumerate(dataset):
    df=df.assign(pfratio=lambda x: x.po2 / x.fio2 )
    dataset[idx]=df
dataset[0].head()

rr    hr   spo2    sbp   dbp  \
subject_id hadm_id icustay_id hours_in                                   
4          185777  294638     5         16.0  82.0   95.0  101.0  61.0   
                              6         18.0  81.0  100.0   97.0  55.0   
                              7         24.0  80.0  100.0  109.0  58.0   
                              8         24.0  85.0   91.0  110.0  56.0   
                              9         24.0  92.0   98.0  110.0  60.0   

                                             temp        hct    hb  platelets  \
subject_id hadm_id icustay_id hours_in                                          
4          185777  294638     5         37.444445  31.799999  10.6      201.0   
                              6         37.444445  31.799999  10.6      201.0   
                              7         36.666698  31.799999  10.6      201.0   
                              8         36.666698  31.799999  10.6      201.0   
                              9         36.666698  31.799999  10.6      201.0   

                                        wbc  ...   mch  mchc   mcv    ph  \
subject_id hadm_id icustay_id hours_in       ...                           
4          185777  294638     5         9.7  ...  30.8  33.2  93.0  7.47   
                              6         9.7  ...  30.8  33.2  93.0  7.47   
                              7         9.7  ...  30.8  33.2  93.0  7.47   
                              8         9.7  ...  30.8  33.2  93.0  7.47   
                              9         9.7  ...  30.8  33.2  93.0  7.47   

                                        hco3   po2  pco2  sao2  fio2  \
subject_id hadm_id icustay_id hours_in                                 
4          185777  294638     5         21.0  74.0  28.0  94.0  0.95   
                              6         21.0  74.0  28.0  94.0  0.95   
                              7         21.0  74.0  28.0  94.0  0.50   
                              8         21.0  74.0  28.0  94.0  0.50   
                              9         21.0  74.0  28.0  94.0  0.50   

                                           pfratio  
subject_id hadm_id icustay_id hours_in              
4          185777  294638     5          77.894738  
                              6          77.894738  
                              7         148.000000  
                              8         148.000000  
                              9         148.000000  

[5 rows x 21 columns]

In [23]:
#apply merge
for idx, df in enumerate(dataset):
    dataset[idx]= pd.merge(df.reset_index(), static_to_keep, on=['subject_id','icustay_id','hadm_id'])
dataset[0].head()

,subject_id,hadm_id,icustay_id,hours_in,rr,hr,spo2,sbp,dbp,temp,...,hco3,po2,pco2,sao2,fio2,pfratio,age,intime,gender_F,gender_M
0,4,185777,294638,5,16.0,82.0,95.0,101.0,61.0,37.444445,...,21.0,74.0,28.0,94.0,0.95,77.894738,47.845047,0,1,0
1,4,185777,294638,6,18.0,81.0,100.0,97.0,55.0,37.444445,...,21.0,74.0,28.0,94.0,0.95,77.894738,47.845047,0,1,0
2,4,185777,294638,7,24.0,80.0,100.0,109.0,58.0,36.666698,...,21.0,74.0,28.0,94.0,0.50,148.000000,47.845047,0,1,0
3,4,185777,294638,8,24.0,85.0,91.0,110.0,56.0,36.666698,...,21.0,74.0,28.0,94.0,0.50,148.000000,47.845047,0,1,0
4,4,185777,294638,9,24.0,92.0,98.0,110.0,60.0,36.666698,...,21.0,74.0,28.0,94.0,0.50,148.000000,47.845047,0,1,0


In [24]:
# add absolute time feature
for idx, df in enumerate(dataset):
    abs_time = (df['intime'] + df['hours_in'])%24
    df.insert(4, 'absolute_time', abs_time)
    df.drop('intime', axis=1, inplace=True)
    df=df[['subject_id', 'hadm_id', 'icustay_id', 'hours_in','absolute_time', 'rr', 'hr', 'spo2', 'sbp', 'dbp', 'temp', 'hct', 'hb',
       'platelets', 'wbc', 'rbc', 'mch', 'mchc', 'mcv', 'ph', 'hco3', 'po2',
       'pco2', 'sao2', 'fio2', 'pfratio', 'age', 'gender_F', 'gender_M']]
    dataset[idx]=df
dataset[0].head()

,subject_id,hadm_id,icustay_id,hours_in,absolute_time,rr,hr,spo2,sbp,dbp,...,ph,hco3,po2,pco2,sao2,fio2,pfratio,age,gender_F,gender_M
0,4,185777,294638,5,5,16.0,82.0,95.0,101.0,61.0,...,7.47,21.0,74.0,28.0,94.0,0.95,77.894738,47.845047,1,0
1,4,185777,294638,6,6,18.0,81.0,100.0,97.0,55.0,...,7.47,21.0,74.0,28.0,94.0,0.95,77.894738,47.845047,1,0
2,4,185777,294638,7,7,24.0,80.0,100.0,109.0,58.0,...,7.47,21.0,74.0,28.0,94.0,0.50,148.000000,47.845047,1,0
3,4,185777,294638,8,8,24.0,85.0,91.0,110.0,56.0,...,7.47,21.0,74.0,28.0,94.0,0.50,148.000000,47.845047,1,0
4,4,185777,294638,9,9,24.0,92.0,98.0,110.0,60.0,...,7.47,21.0,74.0,28.0,94.0,0.50,148.000000,47.845047,1,0


In [26]:
#data loading
COHORT = ["all","micu"]
if use_mean_impute == 1:
    IMPUTE = "icu_ave"
else:
    if FILLNA_CODE == 0:
        IMPUTE = "ref_val"
    if FILLNA_CODE == 1:
        IMPUTE = "bfill_6h"
    if FILLNA_CODE == 2:
        IMPUTE = "no_bfill"
    if FILLNA_CODE == 3:
        IMPUTE = "bfill_6h_drop"
    if FILLNA_CODE == 4:
        IMPUTE = 'dropref'
    

SAMPLING = ["worst","mean","first"]
path = os.path.join("../data/mimic",COHORT[0],IMPUTE,SAMPLING[0]+".h5")
print(path)

../data/mimic/all/dropref/worst.h5


In [27]:
import os.path

for idx, df in enumerate(dataset):
    path = os.path.join("../data/mimic",COHORT[0],IMPUTE,SAMPLING[idx]+".h5")
    print(path)

    df.set_index(['subject_id', 'hadm_id', 'icustay_id', 'hours_in']).to_hdf(path,'vitals_labs')
    Y.set_index(['subject_id', 'hadm_id', 'icustay_id', 'hours_in']).to_hdf(path,'interventions')
    static.to_hdf(path,'patients', format='table')

mICU_list = static[static['first_careunit']=='MICU']['subject_id'].reset_index(drop=True)
Y_micu = Y[Y['subject_id'].isin(mICU_list)].copy()
static_micu = static[static['subject_id'].isin(mICU_list)].copy()
static_to_keep_micu = static_to_keep[static_to_keep['subject_id'].isin(mICU_list)].copy()

for idx, df in enumerate(dataset):
    df=df[df['subject_id'].isin(mICU_list)].copy()
    path = os.path.join("../data/mimic",COHORT[1],IMPUTE,SAMPLING[idx]+".h5")
    print(path)
    
    df.set_index(['subject_id', 'hadm_id', 'icustay_id', 'hours_in']).to_hdf(path,'vitals_labs')
    Y_micu.set_index(['subject_id', 'hadm_id', 'icustay_id', 'hours_in']).to_hdf(path,'interventions')
    static_micu.to_hdf(path,'patients', format='table')

../data/mimic/all/dropref/worst.h5
../data/mimic/all/dropref/mean.h5
../data/mimic/all/dropref/first.h5
../data/mimic/micu/dropref/worst.h5
../data/mimic/micu/dropref/mean.h5
../data/mimic/micu/dropref/first.h5


In [28]:
# import os.path

# DATAFILE = '../data/mimic/'
# if use_mean_impute == 0:
#     name = ['mimic_all_worst.h5','mimic_all_mean.h5','mimic_all_first.h5']
# else:
#     name = ['mimic_all_worst_meanimp.h5','mimic_all_mean_meanimp.h5','mimic_all_first_meanimp.h5']

# for idx, df in enumerate(dataset):
#     path = os.path.join(DATAFILE,name[idx])
#     print(path)
#     df.set_index(['subject_id', 'hadm_id', 'icustay_id', 'hours_in']).to_hdf(path,'vitals_labs')
#     Y.set_index(['subject_id', 'hadm_id', 'icustay_id', 'hours_in']).to_hdf(path,'interventions')
#     static.to_hdf(path,'patients', format='table')

In [29]:
# DATAFILE = '../data/mimic/'
# if use_mean_impute == 0:
#     name_mICU = ['mimic_mICU_worst.h5','mimic_mICU_mean.h5','mimic_mICU_first.h5']
# else:
#     name_mICU = ['mimic_mICU_worst_meanimp.h5','mimic_mICU_mean_meanimp.h5','mimic_mICU_first_meanimp.h5']

# mICU_list = static[static['first_careunit']=='MICU']['subject_id'].reset_index(drop=True)
# Y_micu = Y[Y['subject_id'].isin(mICU_list)].copy()
# static_micu = static[static['subject_id'].isin(mICU_list)].copy()
# static_to_keep_micu = static_to_keep[static_to_keep['subject_id'].isin(mICU_list)].copy()

# for idx, df in enumerate(dataset):
#     df=df[df['subject_id'].isin(mICU_list)].copy()
#     path = os.path.join(DATAFILE,name_mICU[idx])
#     print(path)
#     df.set_index(['subject_id', 'hadm_id', 'icustay_id', 'hours_in']).to_hdf(path,'vitals_labs')
#     Y_micu.set_index(['subject_id', 'hadm_id', 'icustay_id', 'hours_in']).to_hdf(path,'interventions')
#     static_micu.to_hdf(path,'patients', format='table')

In [30]:
# save data to 
#data loading
# if use_mean_impute == 1:
#     IMPUTE = "icu_ave"
# else:
#     if FILLNA_CODE == 0:
#         IMPUTE = "ref_val"
#     if FILLNA_CODE == 1:
#         IMPUTE = "bfill_6h"
#     if FILLNA_CODE == 2:
#         IMPUTE = "no_bfill"
#     if FILLNA_CODE == 3:
#         IMPUTE = "bfill_6h_drop"

path = os.path.join("../data/mimic",COHORT[0],'no_bfill',SAMPLING[0]+".h5")
print(path)
test = pd.read_hdf(path, 'vitals_labs')
print(test.shape)
test.describe()

../data/mimic/all/no_bfill/worst.h5
(1447655, 25)


,absolute_time,rr,hr,spo2,sbp,dbp,temp,hct,hb,platelets,...,ph,hco3,po2,pco2,sao2,fio2,pfratio,age,gender_F,gender_M
count,1.447655e+06,1.447655e+06,1.447655e+06,1.447655e+06,1.447655e+06,1.447655e+06,1.447655e+06,1.447655e+06,1.447655e+06,1.447655e+06,...,1.447655e+06,1.447655e+06,1.447655e+06,1.447655e+06,1.447655e+06,1.447655e+06,1.447655e+06,1.447655e+06,1.447655e+06,1.447655e+06
mean,1.141631e+01,1.917235e+01,8.442563e+01,9.657142e+01,1.221938e+02,6.138696e+01,3.683470e+01,3.227800e+01,1.091254e+01,2.236816e+02,...,7.396281e+00,2.446018e+01,1.046806e+02,4.035354e+01,9.395077e+01,3.349299e-01,3.812726e+02,6.436657e+01,4.568402e-01,5.431598e-01
std,6.864969e+00,6.530110e+00,1.836491e+01,4.018359e+00,2.408287e+01,1.554359e+01,7.218335e-01,5.504246e+00,1.930804e+00,1.164917e+02,...,6.038963e-02,4.439735e+00,5.620624e+01,7.466004e+00,9.273566e+00,1.990485e-01,2.394911e+02,1.752963e+01,4.981339e-01,4.981339e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.600000e+01,0.000000e+00,0.000000e+00,0.000000e+00,...,6.370000e+00,0.000000e+00,3.200000e+01,0.000000e+00,0.000000e+00,2.100000e-01,3.200000e+01,1.802208e+01,0.000000e+00,0.000000e+00
25%,6.000000e+00,1.600000e+01,7.100000e+01,9.500000e+01,1.060000e+02,5.100000e+01,3.638889e+01,2.840000e+01,9.500000e+00,1.480000e+02,...,7.390000e+00,2.200000e+01,8.750000e+01,3.800000e+01,9.700000e+01,2.100000e-01,2.320000e+02,5.254306e+01,0.000000e+00,0.000000e+00
50%,1.100000e+01,1.900000e+01,8.300000e+01,9.700000e+01,1.200000e+02,6.000000e+01,3.677780e+01,3.160000e+01,1.070000e+01,2.090000e+02,...,7.400000e+00,2.400000e+01,8.750000e+01,4.000000e+01,9.700000e+01,2.100000e-01,4.166667e+02,6.638964e+01,0.000000e+00,1.000000e+00
75%,1.700000e+01,2.300000e+01,9.600000e+01,9.900000e+01,1.370000e+02,7.000000e+01,3.722222e+01,3.580000e+01,1.220000e+01,2.750000e+02,...,7.420000e+00,2.700000e+01,1.070000e+02,4.100000e+01,9.700000e+01,4.000000e-01,4.166667e+02,7.871716e+01,1.000000e+00,1.000000e+00
max,2.300000e+01,2.500000e+02,3.000000e+02,1.000000e+02,3.000000e+02,2.980000e+02,4.277780e+01,7.170000e+01,2.210000e+01,2.000000e+03,...,8.400000e+00,5.300000e+01,7.000000e+02,2.000000e+02,1.000000e+02,1.000000e+00,3.333333e+03,1.002809e+02,1.000000e+00,1.000000e+00


In [31]:
test.shape

(1447655, 25)